In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')


# LangChain LLM Wrappers

In [19]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.6)
text="Who is the author of the book Harry Potter?"
print(llm.predict(text))



J.K. Rowling


In [21]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN, repo_id="google/flan-t5-large",model_kwargs={"temperature":0.4,"max_length":264})
output=llm_huggingface.predict("Write a haiku about sadness")
print(output)

sadness is a sour taste


In [22]:
text="Write a haiku about sadness"
print(llm.predict(text))



Tears fall like raindrops
Heavy heart weighs like boulders
Sadness consumes me



# Prompt Template And LLMChain

In [24]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['subject'], template="Write a haiku about {subject}")

prompt_template.format(subject="clocks")

'Write a haiku about clocks'

In [25]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("falafels"))

c:\code\langchain-qna\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




Crispy chickpea balls
Golden brown and full of spice
Satisfy my soul 



# Combining Multiple Chains Using Simple Sequential Chain

In [54]:
question_template=PromptTemplate(input_variables=['area'], template="Give me the name of one concept used in {area}")
question_chain=LLMChain(llm=llm,prompt=question_template)
eli11_template=PromptTemplate(input_variables=['topic'], template="Explain {topic} to an eleven year old")
eli11_chain=LLMChain(llm=llm,prompt=eli11_template)

In [55]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[question_chain, eli11_chain])
print(chain.run("Machine learning"))



Neural networks are like a big group of friends who work together to solve a problem. Each friend has a different job, and they all share information with each other to figure out the best solution. Just like how we ask our friends for help with homework or a game, neural networks ask their friends (or neurons) for help to solve a problem. And the more they work together, the better they get at solving the problem! 


In [32]:
print(chain.run("Aviation"))



Imagine you are blowing up a balloon and then letting it go. As you let go, the air rushes out of the balloon quickly, causing the balloon to fly around the room. This is because of Bernoulli's principle! The air inside the balloon is moving very quickly, so its pressure decreases. This creates a difference in air pressure between the inside and outside of the balloon, causing it to move. This same principle is used in airplanes to create lift and in jet engines to produce thrust. It's like a superpower that helps airplanes fly and move through the air!


# Using Keys in Sequential Chain

In [51]:
question_template=PromptTemplate(input_variables=['area'], template="Give me the name of one concept used in {area}")
question_chain=LLMChain(llm=llm,prompt=question_template, output_key='concept')
eli11_template=PromptTemplate(input_variables=['concept'], template="Explain {concept} to an eleven year old")
eli11_chain=LLMChain(llm=llm,prompt=eli11_template, output_key='explanation')

In [52]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[question_chain, eli11_chain], input_variables=['area'], output_variables=['concept', 'explanation'])

In [53]:
chain({'area':"Aviation"})

{'area': 'Aviation',
 'concept': '\n\nAerodynamics\n',
 'explanation': "\n\nAerodynamics is the study of how air moves around objects, like airplanes, cars, and even birds. It helps us understand how to design things that can move through the air efficiently and safely. By using aerodynamics, we can make things that can fly faster, go further, and use less fuel. It's like figuring out the best way to ride a bike to make it go faster and easier!"}

# Chatmodels With ChatOpenAI


In [56]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [57]:
chatllm=ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.3,model='gpt-3.5-turbo')

c:\code\langchain-qna\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [61]:
chatllm([
SystemMessage(content="You are an AI assitant that gives explanations in such a way that eleven year olds can understand them."),
HumanMessage(content="What is a quantum computer?")
])

AIMessage(content="A quantum computer is a super-duper powerful computer that uses something called quantum mechanics to do really complex calculations. Regular computers use bits to store and process information, which are like tiny switches that can be either on or off, represented by 0s and 1s. But quantum computers use something called qubits, which can be both 0 and 1 at the same time! This is because of a weird property called superposition.\n\nBecause of this superposition thing, quantum computers can do a lot of calculations all at once, which makes them really fast and powerful. They can solve problems that would take regular computers a really long time, like figuring out the best route for a delivery truck or finding new medicines. It's like having a super brain that can think about a million things at once! But quantum computers are still being developed and are not yet as common as regular computers.")

# Prompt Template + LLM +Output Parsers

In [62]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [65]:

class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [63]:
template="Your are a helpful assistant. When the user gives any word, you should generate 5 synonyms in a comma separated list."
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [66]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [67]:
chain.invoke({"text":"kind"})

['generous', ' considerate', ' compassionate', ' benevolent', ' thoughtful']